In [2]:
import requests
import pandas as pd
from datetime import datetime

# Define parameters
DivrtID = "10987"
end_date = datetime.today().strftime("%Y-%m-%d")  # Get today's date in YYYY-MM-DD format

# Construct API URL
api_url = f"https://www.waterrights.utah.gov/dvrtdb/daily-chart.asp?station_id={DivrtID}&end_date={end_date}&f=json"

print(api_url)

# Fetch data from the API
response = requests.get(api_url)

# Check if request was successful
if response.status_code == 200:
    data = response.json()
    
    # Extract metadata fields
    metadata_fields = ["station_id", "system_name", "station_name", "units"]
    metadata = {field: data.get(field, "N/A") for field in metadata_fields}  # Default to "N/A" if missing
    
    # Extract time series data
    if "data" in data:
        df = pd.DataFrame(data["data"], columns=["date", "value"])
        df.rename(columns={"date": "Date", "value": "Value"}, inplace=True)  # Ensure correct column names

        # Add metadata as new columns
        for key, value in metadata.items():
            df[key] = value
        
        # Reorder columns to: station_id, station_name, system_name, units, Date, Value
        column_order = ["station_id", "station_name", "system_name", "units", "Date", "Value"]
        df = df[column_order]

        # Define CSV filename with station_id and date
        csv_filename = f"water_data_{DivrtID}_{end_date}.csv"

        # Save DataFrame to CSV
        df.to_csv(csv_filename, index=False)

        print(f"Data successfully saved to {csv_filename}")
    else:
        print("Error: 'data' key not found in API response.")
else:
    print(f"Error fetching data: {response.status_code}")



https://www.waterrights.utah.gov/dvrtdb/daily-chart.asp?station_id=10987&end_date=2025-04-01&f=json
Data successfully saved to water_data_10987_2025-04-01.csv
